# Quickstart Guide

<div class="alert alert-block alert-info">
    <b>Pre-requisites:</b> CIMS is installed in your active conda environment (<a href="../../docs/INSTALL.md">installation instructions</a>).
</div>

This notebook serves as a brief overview for getting started with CIMS. The diagram below shows the high-level workflow for running a CIMS model. This Quickstart Guide focuses on portions of the workflow that happen inside of Python (shows as a yellow box in this diagram). 

[![](https://mermaid.ink/img/pako:eNpVkEFrwzAMhf-K0bk9rO0ph0KTbacVRgq72D1osbIYHDs4NltW-t-n1DASncR7n4T0btB4TVBAa_1302GI4q1WTnCdZBUII4kzE1a8GktXsd0exfsUO-8yNKbPr4BDtxLnKp_kB1qj_xfk2XK3QHayTMbqlb9f-HtZJ7dyDwv3IF9-Bs8H1zQmG8ercuR0BvI1j5kqK5U8ObTTLy1w2EBPoUejOYDbzCmIHfWkoOBWU4tMKlDuziim6C-Ta6CIIdEG0jA_92yQA-ihaNGOrJI20YdzDvWR7f0PMvluoQ?type=png)](https://mermaid.live/edit#pako:eNpVkEFrwzAMhf-K0bk9rO0ph0KTbacVRgq72D1osbIYHDs4NltW-t-n1DASncR7n4T0btB4TVBAa_1302GI4q1WTnCdZBUII4kzE1a8GktXsd0exfsUO-8yNKbPr4BDtxLnKp_kB1qj_xfk2XK3QHayTMbqlb9f-HtZJ7dyDwv3IF9-Bs8H1zQmG8ercuR0BvI1j5kqK5U8ObTTLy1w2EBPoUejOYDbzCmIHfWkoOBWU4tMKlDuziim6C-Ta6CIIdEG0jA_92yQA-ihaNGOrJI20YdzDvWR7f0PMvluoQ)

## Import CIMS  & other packages

Before any other work can be done, we need to import the CIMS Python package (along with a couple other packages we'll end up using). 

In [ ]:
import CIMS
import pprint as pp

## Validate Model

With the CIMS package loaded, we can validate the model we specified in Excel. This will run some quick checks to ensure we haven't made obvious mistakes when specifying the model in Excel. For example, it will check whether you've specified a fuel node, without also providing a base-year LCC or price. 

Address any problems flagged by the model by adjusting your model in Excel. Re-run the model validator to ensure there are no outstanding problems.    

For more information on using the `ModelValidator` class, checkout the [model validation tutorial](ModelValidation.ipynb). 

In [ ]:
# Spcify the location of the model file
model_file = '../models/CIMS_base model 2.xlsb'

# Create the Model Validator
model_validator = CIMS.ModelValidator(
    infile=model_file, 
    sheet_map={
        'model': ['CIMS', 'CAN', 'BC'],
        'default_param': 'Default values'
    },
    node_col='Branch',
    root_node='CIMS'
)

# Run Validation
model_validator.validate(verbose=True, raise_warnings=False)

## Build Model

There are two steps to building the model. First, we need to create a `ModelReader` object. This object reads in the model file and does some processessing to prepare the description for use as a model. Then, this `ModelReader` object is used to build a `Model` object, which we will be using going forward.

In [ ]:
# Model Reader
years = ['2000', '2005', '2010', '2015', '2020', '2025', '2030']
model_reader = CIMS.ModelReader(
    infile=model_file,
    sheet_map={
        'model': ['CIMS', 'CAN', 'BC'],
        'incompatible': 'Incompatible',
        'default_param': 'Default values'},
    node_col='Branch',
    year_list=years,
    col_list=['Branch', 'Sector', 'Technology', 'Parameter', 'Context', 'Sub_Context', 'Target', 'Source','Unit']+ years,
    sector_list=[],
    root_node='CIMS'
)

In [ ]:
# Model
model = CIMS.Model(model_reader)

## Run Model

Use the `.run()` method to run the model. The year & iteration currently being processed by the model is noted printed to the screen, enabling you to track its progress. Optionally, can specify the maximum number of iterations that can be run in each year, if the model is unable to reach an equilibrium.

In [ ]:
model.run(max_iterations=5)

## Export Results

Once you've run your model, use the `CIMS.log_model()` function to export the results to a CSV file. By default, all model parameters will be written to the CSV. However, using the optional arguments `default_list` or `parameter_list` you can specify particular subsets of parameters to write out. 

Using the `default_list` argument provides users the option to save pre-defined lists of parameters. Currently, two default parameter lists are defined: (1) all (saves all parameters) and (2) slim (saves `new_market_share`, `price`, `competition type`, `service_requested`, and `fcc`). 

Alternatively, the `parameter_list` argument allows users to specify the exact parameters they want to save, by providing a list to the function. 

In [ ]:
# This will save all parameters to my_cims_log.csv
df = CIMS.log_model(model=model, output_file="my_cims_log.csv")

In [ ]:
# This will save 5 parameters (new_market_share, price, competition type, service_requested, and fcc) to my_cims_log.csv
df = CIMS.log_model(model=model, output_file="my_cims_log.csv", default_list="slim")

In [ ]:
# This will save 3 parameters (price, total_market_share, total_cumul_emissions_cost) to my_cims_log.csv. 
df = CIMS.log_model(model=model, output_file="my_cims_log.csv", parameter_list=["price", "total_market_share", "total_cumul_emissions_cost"])

Once you've exported the model to a CSV file, you can open this CSV file in Excel (or your preferred viewer) to do analysis.

## Advanced &mdash; Examine the Results in Python

Some users may prefer to examine model results within Python, rather than exporting the model into a GUI such as Excel. This section provides examples of the questions you can ask by querying the model from inside of Python.

#### How has a fuel's price changed over the years?
This cell will print out the LCC for each year for the specified fuel. The `fuel_of_interest` variable should be the path to the fuel node of interest. 

In [ ]:
fuel_of_interest = 'CIMS.Generic Fuels.Natural Gas' # Change this line to see how other fuels change

print("----- {} -----".format(fuel_of_interest))
for year in model.years:
    lcc = model.get_param('price', fuel_of_interest, year)
    print('{}: {}'.format(year, round(lcc, 2)))


#### How has marketshare changed between years?
This cell will print out the market shares for each technology for each within the specified node. The `node_of_interest` variable should be the path to the node of interest. 

In [ ]:
node_of_interest = 'CIMS.Canada.British Columbia.Residential.Buildings.Ranges' # Change this line to see other nodes

# Printing the Table Header
headers = ('Technology', 'New Market Share', 'Total Market Share')
dash = '-' * 70
cols = '{:<20}{:>25}{:>25}'
print("{}".format(node_of_interest))
print(dash)
print(cols.format(*headers))             
print(dash)

# Printing the Table Content
total_stocks = {}
for year in model.years:
    print(year)
    for tech in model.graph.nodes[node_of_interest][year]['technologies']:
        n_ms = model.get_param('new_market_share', node_of_interest, year, tech)
        t_ms = model.get_param('total_market_share', node_of_interest, year, tech)
        print(cols.format(tech, f"{round(n_ms*100)}%", f"{round(t_ms*100)}%"))             
    print(dash)



#### How to find a Node's Path
This cell retrieves all of the nodes within the graph whose name contains the specified `node_name` . A Node's name is the last element in path, where elements are seperated by `.`. 

In [ ]:
node_name = 'Furnace'

for n in model.graph.nodes:
    n_name = n.split('.')[-1]
    if node_name in n_name:
        print(n)
